In [1]:
import asyncio
import sqlite3

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI



In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

In [3]:
from langchain_openai import ChatOpenAI

llm= ChatOpenAI(
    model="gpt-4.1-mini-2025-04-14",
    api_key=api_key,
)


In [4]:
llm.invoke("hi")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 8, 'total_tokens': 17, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_658b958c37', 'id': 'chatcmpl-Bj2ejhNClFmu9YIroxCaon5jh3GAM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--62b0d1e7-6fd2-4d67-844c-446eca45ae69-0', usage_metadata={'input_tokens': 8, 'output_tokens': 9, 'total_tokens': 17, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.utilities.sql_database import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///ecommerce.db")

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

In [ ]:


# Create the LangChain tool adapter for every tool in the toolkit.
tools = [LangChainToolAdapter(tool) for tool in toolkit.get_tools()]

# Create the chat completion client.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Create the assistant agent.
DB_agent = AssistantAgent(
    "DB_assistant",
    model_client=model_client,
    tools=tools,  # type: ignore
    model_client_stream=True,
    system_message="Your task is convert the user query into SQL query and return Data, Respond with 'TERMINATE' if the task is completed.",
)

# Create termination condition.
termination = TextMentionTermination("TERMINATE")

# Create a round-robin group chat to iterate the single agent over multiple steps.
chat = RoundRobinGroupChat([DB_agent], termination_condition=termination)

# Run the chat.
await Console(chat.run_stream(task="give me trend  sales trends  vs other paramater "))

---------- TextMessage (user) ----------
give me trend  sales trends  vs other paramater 
---------- ToolCallRequestEvent (DB_assistant) ----------
[FunctionCall(id='call_CbDiFU9pSixdAu8S7lb7tuIB', arguments='{}', name='sql_db_list_tables')]
---------- ToolCallExecutionEvent (DB_assistant) ----------
[FunctionExecutionResult(content='customers, order_items, orders, products, suppliers', name='sql_db_list_tables', call_id='call_CbDiFU9pSixdAu8S7lb7tuIB', is_error=False)]
---------- ToolCallSummaryMessage (DB_assistant) ----------
customers, order_items, orders, products, suppliers
---------- ToolCallRequestEvent (DB_assistant) ----------
[FunctionCall(id='call_21XdV8ALEuhpJTI9VD9pt762', arguments='{"table_names": "orders"}', name='sql_db_schema'), FunctionCall(id='call_c9t4S5VHAguaKznaPKVufI4o', arguments='{"table_names": "order_items"}', name='sql_db_schema'), FunctionCall(id='call_kV5YlDUq9ISVEGnbbCodKoMJ', arguments='{"table_names": "products"}', name='sql_db_schema')]
---------- Too

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 16, 11, 51, 12, 868181, tzinfo=datetime.timezone.utc), content='give me trend  sales trends  vs other paramater ', type='TextMessage'), ToolCallRequestEvent(source='DB_assistant', models_usage=RequestUsage(prompt_tokens=0, completion_tokens=0), metadata={}, created_at=datetime.datetime(2025, 6, 16, 11, 51, 13, 916011, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='call_CbDiFU9pSixdAu8S7lb7tuIB', arguments='{}', name='sql_db_list_tables')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='DB_assistant', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 6, 16, 11, 51, 13, 926036, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='customers, order_items, orders, products, suppliers', name='sql_db_list_tables', call_id='call_CbDiFU9pSixdAu8S7lb7tuIB', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSumma

# plot visionaltion

In [21]:
type(plot_fig)

matplotlib.figure.Figure

In [ ]:
import pandas as pd
import io
import base64
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
from typing import List, Tuple, Optional, Union

def create_bar_chart(
    data: List[List[Union[str, int, float]]], 
    title: str = "Bar Chart", 
    color: str = 'skyblue', 
    xlabel: str = "Category", 
    ylabel: str = "Value"
) -> Union[dict]:
    """
    Create a bar chart from list data
    
    Args:
        data: List of [category, value] pairs, e.g., [['A', 23], ['B', 45], ['C', 56]]
              This parameter is required and cannot be None or empty
        title: Title for the chart (default: "Bar Chart")
        color: Bar color (default: 'skyblue')
        xlabel: X-axis label (default: "Category")
        ylabel: Y-axis label (default: "Value")
    
    Returns:
        dict with plot path or error dict with 'error' key
    """
    try:
        if not data:
            return {"error": "Data parameter is required and cannot be empty. Expected format: [['category1', value1], ['category2', value2], ...]"}
        
        if not isinstance(data, list):
            return {"error": "Data must be a list of lists. Expected format: [['category1', value1], ['category2', value2], ...]"}
        
        for i, item in enumerate(data):
            if not isinstance(item, list) or len(item) != 2:
                return {"error": f"Data item at index {i} must be a list with exactly 2 elements [category, value]"}
        
        categories = [str(item[0]) for item in data]
        values = [float(item[1]) for item in data]
        
        plt.figure(figsize=(10, 6))
        plt.bar(categories, values, color=color)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.xlabel(xlabel, fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"plots/bar_chart_{timestamp}.png"
        os.makedirs("plots", exist_ok=True)
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close()
        
        return {
            "status": "success",
            "plot_path": filepath,
            "absolute_path": os.path.abspath(filepath)
        }
    except Exception as e:
        plt.close()
        return {"error": f"Error creating bar chart: {str(e)}"}

def create_line_chart(
    data: List[List[Union[int, float]]], 
    title: str = "Line Chart", 
    color: str = 'blue', 
    xlabel: str = "X", 
    ylabel: str = "Y"
) -> Union[dict]:
    """
    Create a line chart from list data
    
    Args:
        data: List of [x, y] pairs, e.g., [[1, 10], [2, 20], [3, 15]]
              This parameter is required and cannot be None or empty
        title: Title for the chart (default: "Line Chart")
        color: Line color (default: 'blue')
        xlabel: X-axis label (default: "X")
        ylabel: Y-axis label (default: "Y")
    
    Returns:
        dict with plot path or error dict with 'error' key
    """
    try:
        if not data:
            return {"error": "Data parameter is required and cannot be empty. Expected format: [[x1, y1], [x2, y2], ...]"}
        
        if not isinstance(data, list):
            return {"error": "Data must be a list of lists. Expected format: [[x1, y1], [x2, y2], ...]"}
        
        for i, item in enumerate(data):
            if not isinstance(item, list) or len(item) != 2:
                return {"error": f"Data item at index {i} must be a list with exactly 2 numeric elements [x, y]"}
        
        x_values = [float(item[0]) for item in data]
        y_values = [float(item[1]) for item in data]
        
        plt.figure(figsize=(10, 6))
        plt.plot(x_values, y_values, marker='o', color=color, linewidth=2)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.xlabel(xlabel, fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"plots/line_chart_{timestamp}.png"
        os.makedirs("plots", exist_ok=True)
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close()
        
        return {
            "status": "success",
            "plot_path": filepath,
            "absolute_path": os.path.abspath(filepath)
        }
    except Exception as e:
        plt.close()
        return {"error": f"Error creating line chart: {str(e)}"}

def create_histogram(
    data: List[Union[int, float]], 
    bins: int = 20, 
    title: str = "Histogram", 
    color: str = 'steelblue', 
    xlabel: str = "Values"
) -> Union[dict]:
    """
    Create a histogram from list data
    
    Args:
        data: List of numeric values, e.g., [1, 2, 3, 4, 5, 2, 3, 1]
              This parameter is required and cannot be None or empty
        bins: Number of bins for histogram (default: 20)
        title: Title for the chart (default: "Histogram")
        color: Bar color (default: 'steelblue')
        xlabel: X-axis label (default: "Values")
    
    Returns:
        dict with plot path or error dict with 'error' key
    """
    try:
        if not data:
            return {"error": "Data parameter is required and cannot be empty. Expected format: [value1, value2, value3, ...]"}
        
        if not isinstance(data, list):
            return {"error": "Data must be a list of numeric values. Expected format: [value1, value2, value3, ...]"}
        
        numeric_data = [float(x) for x in data]
        
        plt.figure(figsize=(10, 6))
        plt.hist(numeric_data, bins=bins, alpha=0.7, edgecolor='black', color=color)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.xlabel(xlabel, fontsize=12)
        plt.ylabel("Frequency", fontsize=12)
        plt.grid(True, alpha=0.3, axis='y')
        plt.tight_layout()
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"plots/histogram_{timestamp}.png"
        os.makedirs("plots", exist_ok=True)
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close()
        
        return {
            "status": "success",
            "plot_path": filepath,
            "absolute_path": os.path.abspath(filepath)
        }
    except Exception as e:
        plt.close()
        return {"error": f"Error creating histogram: {str(e)}"}

def create_scatter_plot(
    data: List[List[Union[int, float]]], 
    title: str = "Scatter Plot", 
    color: str = 'red', 
    xlabel: str = "X", 
    ylabel: str = "Y"
) -> Union[dict]:
    """
    Create a scatter plot from list data
    
    Args:
        data: List of [x, y] pairs, e.g., [[1, 2], [3, 4], [5, 6]]
              This parameter is required and cannot be None or empty
        title: Title for the chart (default: "Scatter Plot")
        color: Point color (default: 'red')
        xlabel: X-axis label (default: "X")
        ylabel: Y-axis label (default: "Y")
    
    Returns:
        dict with plot path or error dict with 'error' key
    """
    try:
        if not data:
            return {"error": "Data parameter is required and cannot be empty. Expected format: [[x1, y1], [x2, y2], ...]"}
        
        if not isinstance(data, list):
            return {"error": "Data must be a list of lists. Expected format: [[x1, y1], [x2, y2], ...]"}
        
        for i, item in enumerate(data):
            if not isinstance(item, list) or len(item) != 2:
                return {"error": f"Data item at index {i} must be a list with exactly 2 numeric elements [x, y]"}
        
        x_values = [float(item[0]) for item in data]
        y_values = [float(item[1]) for item in data]
        
        plt.figure(figsize=(10, 6))
        plt.scatter(x_values, y_values, alpha=0.7, color=color, s=50)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.xlabel(xlabel, fontsize=12)
        plt.ylabel(ylabel, fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"plots/scatter_plot_{timestamp}.png"
        os.makedirs("plots", exist_ok=True)
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close()
        
        return {
            "status": "success",
            "plot_path": filepath,
            "absolute_path": os.path.abspath(filepath)
        }
    except Exception as e:
        plt.close()
        return {"error": f"Error creating scatter plot: {str(e)}"}

def create_pie_chart(
    data: List[List[Union[str, int, float]]], 
    title: str = "Pie Chart", 
    colors: Optional[List[str]] = None
) -> Union[dict]:
    """
    Create a pie chart from list data
    
    Args:
        data: List of [label, value] pairs, e.g., [['A', 25], ['B', 35], ['C', 20]]
              This parameter is required and cannot be None or empty
        title: Title for the chart (default: "Pie Chart")
        colors: List of colors for slices (default: None for automatic colors)
    
    Returns:
        dict with plot path or error dict with 'error' key
    """
    try:
        if not data:
            return {"error": "Data parameter is required and cannot be empty. Expected format: [['label1', value1], ['label2', value2], ...]"}
        
        if not isinstance(data, list):
            return {"error": "Data must be a list of lists. Expected format: [['label1', value1], ['label2', value2], ...]"}
        
        for i, item in enumerate(data):
            if not isinstance(item, list) or len(item) != 2:
                return {"error": f"Data item at index {i} must be a list with exactly 2 elements [label, value]"}
        
        labels = [str(item[0]) for item in data]
        values = [float(item[1]) for item in data]
        
        plt.figure(figsize=(8, 8))
        plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
        plt.title(title, fontsize=14, fontweight='bold')
        plt.axis('equal')
        plt.tight_layout()
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filepath = f"plots/pie_chart_{timestamp}.png"
        os.makedirs("plots", exist_ok=True)
        plt.savefig(filepath, dpi=300, bbox_inches='tight')
        plt.close()
        
        return {
            "status": "success",
            "plot_path": filepath,
            "absolute_path": os.path.abspath(filepath)
        }
    except Exception as e:
        plt.close()
        return {"error": f"Error creating pie chart: {str(e)}"}


In [ ]:
tools=[create_line_chart , create_pie_chart , create_scatter_plot, create_histogram, create_bar_chart]
# Create the chat completion client.
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

# Create the assistant agent.
agent = AssistantAgent(
    "plot_genaratewr",
    model_client=model_client,
    tools=tools,  # type: ignore
    model_client_stream=True,
    system_message="Create the best sutable chat from  create_line_chart , create_pie_chart , create_scatter_plot, create_histogram, create_bar_chart based given data .  after complete of plot call say 'TERMINATE' if the task is completed. ",
    reflect_on_tool_use=False
)

# Create termination condition.
termination = TextMentionTermination("TERMINATE")

# Create a round-robin group chat to iterate the single agent over multiple steps.
chat = RoundRobinGroupChat([agent], termination_condition=termination)

# Run the chat.
await Console(chat.run_stream(task="""--
[('2023-06-01', 6), ('2023-06-02', 2), ('2023-06-03', 1), ('2023-06-04', 2), ('2023-06-05', 2), ('2023-06-10', 2), ('2023-06-15', 2), ('2023-06-20', 2), ('2023-06-25', 3), ('2023-06-30', 1), ('2023-07-05', 1), ('2023-07-10', 2)]
[('2023-06', 23), ('2023-07', 3)]  daywise """))




---------- TextMessage (user) ----------
--
[('2023-06-01', 6), ('2023-06-02', 2), ('2023-06-03', 1), ('2023-06-04', 2), ('2023-06-05', 2), ('2023-06-10', 2), ('2023-06-15', 2), ('2023-06-20', 2), ('2023-06-25', 3), ('2023-06-30', 1), ('2023-07-05', 1), ('2023-07-10', 2)]
[('2023-06', 23), ('2023-07', 3)]  daywise 
